In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader  
from torchvision import datasets, models, transforms
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.metrics import precision_score, recall_score

In [2]:
# Definindo dispositivo de hardware
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [3]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)), # Ajuste o tamanho conforme necessário
    transforms.RandomResizedCrop (224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]) #é Normalização de imagem em tons de cinza
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)), # Ajuste o tamanho conforme necessário
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]) # Normalização de imagem em tons de cinza
])

In [4]:
train_dir = "/Datafol/trainfol/"
test_dir = "/Datafol/testfol/"
train_classa_dir = "/Datafol/trainfol/classafol/"
train_classb_dir = "/Datafol/trainfol/classbfol/"
test_classa_dir = "/Datafol/testfol/classafol/"
test_classb_dir = "/Datafol/testfol/classbfol/"

In [5]:
test_dir = ('/home/arthur/Projetos/SPLIT_IMG/test')
train_dir = ('/home/arthur/Projetos/SPLIT_IMG/train')
train_dataset = datasets.ImageFolder(train_dir, transforms_train)
test_dataset = datasets.ImageFolder(test_dir, transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=12, shuffle=True, num_workers=1)

In [6]:
print('Treinar o tamanho do conjunto de dado', len(train_dataset))
print('Testar o tamanho do conjunto de dados', len(test_dataset))
class_names = train_dataset.classes
print('Nomes de classe:', class_names)

Treinar o tamanho do conjunto de dado 4487
Testar o tamanho do conjunto de dados 2335
Nomes de classe: ['-1', '-2', '-3', '0', '1', '2', '3']


In [7]:
model = models.resnet18(pretrained=True) # carrregando rede. resnet18 model
num_features = model.fc.in_features # extraindo features da camada fc
model.fc = nn.Linear(num_features, 7) # é numero de classes

model = model.to(device)

criterion = nn.CrossEntropyLoss() #(set. loss: function)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/arthur/anaconda3/envs/skincancer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/arthur/anaconda3/envs/skincancer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
train_losses = []
train_accs = []
train_precisions = []
train_recalls = []
test_losses = []
test_accs = []
test_precisions = []
test_recalls = []
num_epochs = 100 # define o número de epochs
start_time = time.time() # calcular o tempo de cada época

In [9]:
import time

train_losses = [] 
train_accs = []
test_losses = []
test_accs = []
num_epochs = 100  # seta numero de epochs
start_time = time.time() # calcular tempo de cada epoca
for epoch in range(num_epochs): # loop para rodar as epocas
    print("Epoch {} running".format(epoch)) # printar epoca rodando
    """ Training Phase """
    model.train()    # treinar modelo
    running_loss = 0.   # zerando metricas
    running_corrects = 0 # zerando metricas
    # loop de imagem no treino, classificando imagem pelo rotulo
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # passando as entradas e obtendo resultado
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # obtendo valor da loss e atualizando os pesos da rede
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_dataset)
    train_losses.append(epoch_loss)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    train_accs.append(epoch_acc)
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))
    
    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(test_dataset)
        test_losses.append(epoch_loss) 
        epoch_acc = running_corrects / len(test_dataset) * 100.
        test_accs.append(epoch_acc)
        print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 0 running
[Train #0] Loss: 1.2692 Acc: 52.1729% Time: 29.3587s
[Test #0] Loss: 0.9397 Acc: 63.9400% Time: 36.8515s
Epoch 1 running
[Train #1] Loss: 1.0018 Acc: 62.4916% Time: 63.6849s
[Test #1] Loss: 0.8851 Acc: 67.0236% Time: 70.4038s
Epoch 2 running
[Train #2] Loss: 0.9492 Acc: 64.4974% Time: 97.3059s
[Test #2] Loss: 0.9782 Acc: 65.6531% Time: 104.3053s
Epoch 3 running
[Train #3] Loss: 0.8734 Acc: 67.3947% Time: 131.6143s
[Test #3] Loss: 0.6923 Acc: 74.6467% Time: 138.6102s
Epoch 4 running
[Train #4] Loss: 0.8140 Acc: 69.8908% Time: 166.1383s
[Test #4] Loss: 0.7340 Acc: 72.8051% Time: 172.9590s
Epoch 5 running
[Train #5] Loss: 0.7732 Acc: 70.9605% Time: 200.7756s
[Test #5] Loss: 0.6240 Acc: 77.2163% Time: 207.6953s
Epoch 6 running
[Train #6] Loss: 0.7223 Acc: 73.2115% Time: 235.7910s
[Test #6] Loss: 0.6768 Acc: 75.4176% Time: 242.8536s
Epoch 7 running
[Train #7] Loss: 0.6966 Acc: 74.4595% Time: 270.8040s
[Test #7] Loss: 0.6435 Acc: 77.6445% Time: 277.5705s
Epoch 8 running
[Trai

In [10]:
torch.save(model.state_dict(), 'model.pth')